<a href="https://colab.research.google.com/github/swethasrihari/NLP/blob/main/FakeNewsGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake News Detection using GPT-2 model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
training_path ='/content/drive/MyDrive/Colab Notebooks/NLP-Project-FakeNewsDataset/train.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/project-llama - Sheet1 - project-llama - Sheet1.csv'

In [ ]:
pip install transformers

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, TFGPT2Model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
# Load data
train_data = pd.read_csv(training_path)
test_data = pd.read_csv(test_path)
train_data.dropna()
print(test_data.head)

<bound method NDFrame.head of     Unnamed: 0                                               News  label  \
0            1  Says Vice President Joe Biden "admits that the...      0   
1            2  Active duty males in the military are twice as...      1   
2            3  Texas families have kept more than $10 billion...      1   
3            4  Says bag litter increased after San Francisco ...      0   
4            5  Across the country, $174 billion of the Recove...      1   
..         ...                                                ...    ...   
59          60  The American Conservative Union gives [Rep. Ti...      0   
60          61  Says U.S. Sen Sherrod Brown promised that the ...      0   
61          62         McCain Trying to Have it Both Ways on Iraq      0   
62          63  Says President Obamas new immigration plan is ...      0   
63          64       Nearly 20% of our residents are born abroad.      1   

    predicted label -LLAMA  
0                        0  

In [ ]:
# Define function to extract required columns
def extract_required_columns_into_dataframe(dataset):
    label = dataset['label'].tolist()
    news = dataset['text'].tolist()
    # putting label and news in a data frame
    data = {"news": news, "label": label}
    df = pd.DataFrame(data)
    return df

In [ ]:
trainingSet = extract_required_columns_into_dataframe(train_data)
testingSet= pd.DataFrame(test_data)

,Unnamed: 0,News,label,predicted label -LLAMA
0,1,"Says Vice President Joe Biden ""admits that the...",0,0
1,2,Active duty males in the military are twice as...,1,0
2,3,Texas families have kept more than $10 billion...,1,1
3,4,Says bag litter increased after San Francisco ...,0,1
4,5,"Across the country, $174 billion of the Recove...",1,0
...,...,...,...,...
59,60,The American Conservative Union gives [Rep. Ti...,0,1
60,61,Says U.S. Sen Sherrod Brown promised that the ...,0,0
61,62,McCain Trying to Have it Both Ways on Iraq,0,0
62,63,Says President Obamas new immigration plan is ...,0,0


In [ ]:
print(trainingSet.shape)
print(testingSet.shape)

(20800, 2)
(64, 1)


In [ ]:
for news in testingSet['news']:
  print(news)

Says Vice President Joe Biden "admits that the American people are being scammed" with the economic stimulus package.
Active duty males in the military are twice as likely to develop prostate cancer than their civilian counterparts.
Texas families have kept more than $10 billion in their family budgets since we successfully fought to restore Texas sales tax deduction a decade ago.
Says bag litter increased after San Francisco banned single-use shopping bags.
Across the country, $174 billion of the Recovery Act have been committed in its first 130 days.
A strong bipartisan majority in the House of Representatives voted to defund Obamacare.
The decision for the United States to buy heavy water from Iran will directly subsidize Irans nuclear program.
When Atlanta Police Chief George Turner was interim head of the department, overall crime fell 14 percent and violent crime dropped 22.7 percent.
There is no record of congresswoman Betty Sutton ... ever holding a single in-person town hall m

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def tokenizeNews(tokenizer, news):
   tokenized_news = news.apply(lambda x: tokenizer.encode(str(x), max_length=120, truncation=True))
   return tokenized_news

trainingSet['tokenized_news'] = tokenizeNews(tokenizer, trainingSet['news'])
testingSet['tokenized_news'] = tokenizeNews(tokenizer, testingSet['news'])

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
print(trainingSet['tokenized_news'])
print(trainingSet.shape, testingSet.shape)

0        [18102, 1897, 317, 485, 25, 775, 31279, 447, 2...
1        [23921, 651, 262, 4203, 534, 1204, 13332, 262,...
2        [5195, 262, 14056, 24213, 3497, 921, 46223, 32...
3        [53, 4921, 1315, 7511, 1547, 13095, 554, 14206...
4        [18557, 220, 198, 2025, 10704, 2415, 468, 587,...
                               ...                        
20795    [49, 11463, 309, 13, 314, 13, 49975, 319, 2042...
20796    [2215, 262, 3469, 4696, 17832, 2626, 284, 262,...
20797    [464, 50048, 447, 247, 82, 286, 1909, 6348, 42...
20798    [34259, 46, 11, 3284, 1675, 9340, 42945, 1475,...
20799    [220, 3271, 47528, 318, 281, 1772, 11, 11276, ...
Name: tokenized_news, Length: 20800, dtype: object
(20800, 3) (64, 2)


In [ ]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(trainingSet['tokenized_news'], trainingSet['label'], test_size=0.2, random_state=42)

In [ ]:
# Convert to numpy arrays
x_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=120)
x_val = tf.keras.preprocessing.sequence.pad_sequences(X_val, padding='post', maxlen=120)
x_test = tf.keras.preprocessing.sequence.pad_sequences(testingSet['tokenized_news'], padding='post', maxlen=120)

In [ ]:
# Loading GPT-2 model and adding a classification head on top of it
model = TFGPT2Model.from_pretrained("gpt2", output_hidden_states=True)
inputs = tf.keras.layers.Input(shape=(120,), dtype=tf.int32)
outputs = model(inputs)['last_hidden_state']
pooled_output = tf.reduce_mean(outputs, axis=1)
output = tf.keras.layers.Dense(1, activation='sigmoid')(pooled_output)
classification_model = tf.keras.models.Model(inputs=inputs, outputs=output)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [ ]:
# Compile and train the model
classification_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

In [ ]:
classification_model.fit(x=x_train, y=y_train, epochs=1, batch_size=32, validation_data=(x_val, y_val))

520/520 [==============================] - 474s 852ms/step - loss: 0.0661 - accuracy: 0.9760 - val_loss: 0.0081 - val_accuracy: 0.9976


In [ ]:
# Predict and evaluate
y_pred_probs = classification_model.predict(x_test)
y_pred_labels = (y_pred_probs > 0.5).astype(int)

2/2 [==============================] - 1s 420ms/step


In [ ]:
y_test = testingSet['label']

report = classification_report(y_test, y_pred_labels)
print("\n Classification Report for GPT2 model using Liar dataset\n",report)


 Classification Report for GPT2 model using Liar dataset
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.39      1.00      0.56        25

    accuracy                           0.39        64
   macro avg       0.20      0.50      0.28        64
weighted avg       0.15      0.39      0.22        64



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
